# 1 Data Preparation and Loading

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/car_vs_ambulance_classification_ensemble_cnn'

In [2]:
# automatically reload all modules before executing
%load_ext autoreload
%autoreload 2 

In [3]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.config.list_physical_devices('GPU'), tf.__version__

2024-10-14 15:26:33.049450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 15:26:33.133986: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 15:26:33.156639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 15:26:33.314568: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1728919596.072207     707 cuda_executor.c

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.17.0')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import utils
from src.utils.image_data_loader import ImageDataLoader

In [5]:
utils.utils.gpu_clean_up()

In [6]:
PARAMS_YAML = 'params.yaml'
params_config = utils.utils.read_yaml(yaml_path=PARAMS_YAML)
params_config

2024-10-14 15:26:36,921 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully


ConfigBox({'base': {'random_seed': 42}, 'data_split': {'test_size': 0.2}, 'train': {'image_size': 224, 'batch_size': 16, 'loss': 'binary_crossentropy', 'if_fine_tune': True, 'fine_tune_args': {'models': ['lenet', 'mini_vgg_net', 'shallownet'], 'lr': {'max': 0.01, 'min': 0.0001}, 'epochs': 2, 'optimizers': ['adam', 'sgd']}, 'lenet': {'base_lr': 0.01, 'epochs': 2}, 'mini_vgg_net': {'base_lr': 0.01, 'epochs': 2}, 'shallownet': {'base_lr': 0.01, 'epochs': 2}}, 'evaluate': {'metrics': ['accuracy', 'precision', 'recall'], 'reports_dir': 'src/reports', 'results_dir': 'src/reports/results', 'confusion_matrix_image': 'confusion_matrix.png'}, 'paths': {'dataset_url': 'https://www.kaggle.com/datasets/amansingh0000000/image-classification-and-yolo-dataset/data', 'raw_dataset_dir': 'dataset/raw', 'dataset_path': 'dataset/image-classification-and-yolo-dataset/Ambulance_Car', 'preprocess_dataset': 'dataset/preprocess', 'train_dir': 'dataset/raw/image-classification-and-yolo-dataset/Ambulance_Car/Trai

In [7]:
tf.random.set_seed(params_config.base.random_seed)

In [8]:
image_loader = ImageDataLoader(params_yaml=PARAMS_YAML)
train_ds, val_ds = image_loader.load_image_dataset()

2024-10-14 15:26:36,994 — utils.utils — INFO — yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully


Found 2227 files belonging to 2 classes.


I0000 00:00:1728919597.039970     707 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728919597.042341     707 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728919597.044940     707 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728919597.174164     707 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 557 files belonging to 2 classes.
2024-10-14 15:26:38,059 — src.utils.image_data_loader — INFO — load_image_dataset are prefetched
2024-10-14 15:26:38,059 — src.utils.image_data_loader — INFO — load_image_dataset completed


# 2 Model Training

In [9]:
from models import lenet, mini_vgg_net, shallownet
from tensorflow import keras
from tensorflow.keras import optimizers
import json

In [10]:
with open(params_config.paths.tune_hyperparameters_path, 'r') as f:
    tuned_hyperparams = json.load(f)
tuned_hyperparams

{'lenet': {'optimizer': 'sgd', 'learning_rate': 0.000505466655750672},
 'mini_vgg_net': {'optimizer': 'adam', 'learning_rate': 0.0003497444477863496},
 'shallownet': {'optimizer': 'sgd', 'learning_rate': 0.000505466655750672}}

In [11]:
optimizers_dict = {
    'adam': optimizers.Adam,
    'sgd': optimizers.SGD,
    'rmsprop': optimizers.RMSprop,
}

### LeNet Train

In [12]:
lenet_model = lenet.LeNet(optimizer_name='adam',
                          image_size=params_config.train.image_size,
                          loss=params_config.train.loss,
                          batch_size=params_config.train.batch_size,
                          epochs=params_config.train.lenet.epochs,
                          learning_rate=params_config.train.lenet.base_lr,
                          metrics=params_config.evaluate.metrics)
lenet_model.build()
lenet_model.get_summary()

2024-10-14 15:26:38,547 — models.lenet — INFO — LeNet model built successfully.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 20)   │         1,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 20)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 20)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 50)   │        25,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │    78,400,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,427,571 (299.18 MB)

 Trainable params: 78,427,571 (299.18 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
lenet_model.model_compile()

2024-10-14 15:26:38,600 — models.lenet — INFO — Selecting optimizer adam


In [14]:
# train the network
lenet_history = lenet_model.train(train_data=train_ds,
                                  val_data=val_ds)

2024-10-14 15:26:38,640 — models.lenet — INFO — Selecting optimizer adam


Epoch 1/2


I0000 00:00:1728919601.088644    1241 service.cc:146] XLA service 0x7c753c0039c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728919601.088685    1241 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-10-14 15:26:41.135578: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-14 15:26:41.306131: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


 12/140 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6864 - loss: 4594.9644 - precision: 0.8671 - recall: 0.7480

I0000 00:00:1728919605.138658    1241 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 35/140 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.7461 - loss: 2532.9963 - precision: 0.8547 - recall: 0.8447

2024-10-14 15:26:47.610364: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_1', 36 bytes spill stores, 36 bytes spill loads



140/140 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.8002 - loss: 945.8698 - precision: 0.8424 - recall: 0.9385 - val_accuracy: 0.8618 - val_loss: 0.4030 - val_precision: 0.8618 - val_recall: 1.0000
Epoch 2/2
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8427 - loss: 0.4352 - precision: 0.8427 - recall: 1.0000 - val_accuracy: 0.8618 - val_loss: 0.4156 - val_precision: 0.8618 - val_recall: 1.0000
2024-10-14 15:26:53,137 — models.lenet — INFO — Training completed.


### MiniVGGNet Train

In [15]:
mini_vgg_net_model = mini_vgg_net.MiniVGGNet(optimizer_name='adam',
                                             image_size=params_config.train.image_size,
                                             loss=params_config.train.loss,
                                             batch_size=params_config.train.batch_size,
                                             epochs=params_config.train.mini_vgg_net.epochs,
                                             learning_rate=params_config.train.mini_vgg_net.base_lr,
                                             metrics=params_config.evaluate.metrics)
mini_vgg_net_model.build()
mini_vgg_net_model.get_summary()

2024-10-14 15:26:53,276 — models.mini_vgg_net — INFO — MiniVGGNet model built successfully.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │   102,760,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,829,857 (392.26 MB)

 Trainable params: 102,828,449 (392.26 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [16]:
# train the network
mini_vgg_net_model.model_compile()
mini_vgg_net_history = mini_vgg_net_model.train(train_data=train_ds,
                                                val_data=val_ds)

2024-10-14 15:26:53,325 — models.mini_vgg_net — INFO — Selecting optimizer adam
2024-10-14 15:26:53,330 — models.mini_vgg_net — INFO — Selecting optimizer adam
Epoch 1/2
 14/140 ━━━━━━━━━━━━━━━━━━━━ 40s 323ms/step - accuracy: 0.6284 - loss: 1.0760 - precision: 0.9316 - recall: 0.6219

2024-10-14 15:27:06.968582: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_3', 160 bytes spill stores, 160 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_6', 44 bytes spill stores, 44 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_2', 28 bytes spill stores, 28 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_7', 44 bytes spill stores, 44 bytes spill loads



140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 69ms/step - accuracy: 0.7398 - loss: 0.7734 - precision: 0.8630 - recall: 0.8261 - val_accuracy: 0.8618 - val_loss: 7.8437 - val_precision: 0.8618 - val_recall: 1.0000
Epoch 2/2
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.8250 - loss: 0.4756 - precision: 0.8751 - recall: 0.9264 - val_accuracy: 0.1795 - val_loss: 15.3167 - val_precision: 0.9600 - val_recall: 0.0500
2024-10-14 15:27:16,692 — models.mini_vgg_net — INFO — Training completed.


### ShallowNet Train

In [17]:
shallownet_model = shallownet.ShallowNet(optimizer_name='sgd',
                                         image_size=params_config.train.image_size,
                                         loss=params_config.train.loss,
                                         batch_size=params_config.train.batch_size,
                                         epochs=params_config.train.shallownet.epochs,
                                         learning_rate=params_config.train.shallownet.base_lr,
                                         metrics=params_config.evaluate.metrics)
shallownet_model.build()
shallownet_model.get_summary()

2024-10-14 15:27:16,732 — models.shallownet — INFO — ShallowNet model built successfully.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1605632)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │     1,605,633 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,606,529 (6.13 MB)

 Trainable params: 1,606,529 (6.13 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# train the network
shallownet_model.model_compile()
shallownet_model_history = shallownet_model.train(train_data=train_ds,
                                                  val_data=val_ds)

2024-10-14 15:27:16,770 — models.shallownet — INFO — Selecting optimizer sgd
2024-10-14 15:27:16,774 — models.shallownet — INFO — Selecting optimizer sgd
Epoch 1/2
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7619 - loss: 297120.3125 - precision: 0.8161 - recall: 0.9148 - val_accuracy: 0.8564 - val_loss: 0.5971 - val_precision: 0.8610 - val_recall: 0.9937
Epoch 2/2
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8401 - loss: 0.5797 - precision: 0.8495 - recall: 0.9816 - val_accuracy: 0.8402 - val_loss: 0.5306 - val_precision: 0.8600 - val_recall: 0.9729
2024-10-14 15:27:19,981 — models.shallownet — INFO — Training completed.
